<a href="https://colab.research.google.com/github/JstnClmnt/NLP-Text-Summarization/blob/master/NLP_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import re
np.random.seed(1)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import pandas as pd
df=pd.read_csv("/content/drive/Team Drives/AI Lords/NLP-Text-Summarization/data/news_summary_more.csv",encoding="latin-1")
df.head()


,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [0]:
df['text'] = df['text'].str.replace("[^a-zA-Z#]"," ")
df['headlines'] = df['headlines'].str.replace("[^a-zA-Z#]"," ")
df=df.dropna()

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df["text"], df["headlines"], test_size=0.20, random_state=42)

print(X_train[0])

Saurav Kant  an alumnus of upGrad and IIIT B s PG Program in Machine learning and Artificial Intelligence  was a Sr Systems Engineer at Infosys with almost   years of work experience  The program and upGrad s     degree career support helped him transition to a Data Scientist at Tech Mahindra with     salary hike  upGrad s Online Power Learning has powered   lakh  careers 


In [6]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

X_train_tokenized=[word_tokenize(i)  for i in X_train]
X_test_tokenized=[word_tokenize(i) for i in X_test]
print(X_train_tokenized[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['Hollywood', 'actress', 'Kate', 'Winslet', 'has', 'joined', 'the', 'cast', 'of', 'Titanic', 'director', 'James', 'Cameron', 's', 'upcoming', 'films', 'in', 'the', 'Avatar', 'franchise', 'making', 'this', 'their', 'first', 'venture', 'together', 'after', 'years', 'of', 'Titanic', 's', 'release', 'Cameron', 'said', 'Kate', 'and', 'I', 'have', 'been', 'looking', 'for', 'something', 'to', 'do', 'together', 'since', 'our', 'collaboration', 'on', 'Titanic', 'I', 'can', 't', 'wait', 'to', 'see', 'her', 'bring', 'the', 'character', 'of', 'Ronal', 'to', 'life']


In [7]:
y_train_tokenized=[word_tokenize(i) for i in y_train]
y_test_tokenized=[word_tokenize(i) for i in y_test]
print(y_train_tokenized[0])

['Kate', 'Winslet', 'to', 'work', 'with', 'Titanic', 'maker', 'after', 'years']


In [0]:
words = set([])
for s in X_train_tokenized:
    for w in s:
      if len(w)>1:
        words.add(w.lower())
for s in y_train_tokenized:
    for w in s:
      if len(w)>1:
        words.add(w.lower())
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs

In [9]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2019-05-18 17:25:35--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-05-18 17:25:35--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove.6B.zip.1       18%[==>                 ] 152.74M  8.24MB/s    eta 75s    ^C


In [10]:
!unzip "glove.6B.zip"

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt',encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [10]:
train_sentences_X, test_sentences_X,train_sentences_Y,test_sentences_Y = [], [], [], []

EMB_DIM=300
num_words=len(word2index)+1
print("Number of Words:"+str(num_words))

for s in X_train_tokenized:
    s_int = []
    for w in s:
      if len(w)>1:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
    
for s in X_test_tokenized:
    s_int = []
    for w in s:
      if len(w)>1:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
  
for s in y_train_tokenized:
    s_int = []
    for w in s:
      if len(w)>1:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_Y.append(s_int)
    
for s in y_test_tokenized:
    s_int = []
    for w in s:
       if len(w)>1:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_Y.append(s_int)


MAX_LENGTH = len(max(train_sentences_X, key=len))
print("Max Length: "+str(MAX_LENGTH))  # 271

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
#y_train=to_categorical(y_train)
#y_test=to_categorical(y_test)
print(train_sentences_X[0])
print(train_sentences_Y[0])

Number of Words:71055
Max Length: 68
[31734 49094 59050 13417 52178 25965  4519 40091 27194 22857 17123 36915
 44410 53024 65963 43275  4519 41033 23617 25350 15356 33723 68529 55323
 64066  1123 65210 27194 22857 56145 44410 51544 59050 33050 65334 60021
 21375 64694 53027 66328 48125 64066 25685 68030 61547  4410 22857 45730
 40197 66328 15501  2226  4957  4519 31474 27194  8317 66328 57828     0
     0     0     0     0     0     0     0     0]
[59050, 13417, 66328, 51625, 43881, 22857, 56598, 1123, 65210]


In [11]:
embedding_matrix=np.zeros((num_words,EMB_DIM))
#print(word2index)
numNoEmb=0
for word,i in word2index.items():
    if i>num_words:
        continue
    embedding_vector=embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector
    else:
      numNoEmb+=1
print(numNoEmb)

15994


In [12]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Input,Dense, CuDNNLSTM,LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation,Dropout,Add
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import Constant
import numpy as np
from tensorflow.keras.utils import plot_model

HIDDEN_UNITS=256
"""
Bidirectional LSTM: Others Inspired Encoder-Decoder-seq2seq
"""
encoder_inputs = Input(shape=(MAX_LENGTH,))
encoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_LSTM_R = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

"""
decoder
"""
decoder_inputs = Input(shape=(MAX_LENGTH,))
decoder_embedding = Embedding(num_words,EMB_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_LENGTH,trainable=False,mask_zero=True)(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 
decoder_dense = Dense(num_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model= tf.keras.Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.0001),    
              metrics=["accuracy"])
print(model.summary())
plot_model(model, to_file='model_seq2seq.png')

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 68, 300)      21316500    input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 68)           0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 570368      embedding[0][0]                  
_____________________________________

In [0]:
num_samples = len(train_sentences_Y)
decoder_output_data = np.zeros((num_samples, MAX_LENGTH, num_words), dtype="int32")